<a href="https://colab.research.google.com/github/SoheliPaul/LLM/blob/main/2348561_LLM_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.3/277.3 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 

In [2]:
!pip install wandb

In [3]:
import numpy as np
import pandas as pd
import os
import wandb
import json
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [4]:
# Loading the JSON data from a file
with open('/content/drive/MyDrive/DATASETS/LLM_lab1_SQUAD data/train-v1.1.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [5]:
# Initialize an empty list to store the converted data
converted_data = []

# Iterate through each item in your data
for item in data['data']:
    # Extract context and QA pairs
    context = item['paragraphs'][0]['context']
    qas = item['paragraphs'][0]['qas']

    # Initialize an empty list to store QA pairs for this context
    qa_list = []

    # Iterate through each QA pair
    for qa in qas:
        # Extract question, answer, answer start position, and ID
        question = qa['question']
        answers = qa['answers']
        if answers:
            answer_text = answers[0]['text']
            answer_start = answers[0]['answer_start']
            qa_id = qa['id']

            # Create a QA dictionary in the desired format
            qa_dict = {
                'id': qa_id,
                'is_impossible': False,  # Assuming all answers are possible in your data
                'question': question,
                'answers': [
                    {
                        'text': answer_text,
                        'answer_start': answer_start,
                    }
                ],
            }

            # Append the QA dictionary to the QA list
            qa_list.append(qa_dict)

    # Create a context dictionary with the QA list
    context_dict = {
        'context': context,
        'qas': qa_list,
    }

    # Append the context dictionary to the converted data list
    converted_data.append(context_dict)

# Print only the first 3 items of the converted data list
print(json.dumps(converted_data[:3], indent=4))

[
    {
        "context": "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend \"Venite Ad Me Omnes\". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",
        "qas": [
            {
                "id": "5733be284776f41900661182",
                "is_impossible": false,
                "question": "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?",
                "answers": [
                   

In [6]:
import random

# Randomly sample 100 data samples from eval_data for validation
eval_data_subset = random.sample(converted_data, k=100)

# Save the randomly sampled validation data to a JSON file
output_file = 'random_validation_data.json'
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(eval_data_subset, outfile, ensure_ascii=False, indent=4)

print(f"Randomly sampled validation data saved to '{output_file}'.")

Randomly sampled validation data saved to 'random_validation_data.json'.


In [7]:
model_type="bert"

In [8]:
if model_type == "bert":
    model_name = "bert-base-cased"
    print("Your Selected Model Name is : ", model_name)

elif model_type == "roberta":
    model_name = "roberta-base"
    print("Your Selected Model Name is : ", model_name)

elif model_type == "distilbert":
    model_name = "distilbert-base-cased"
    print("Your Selected Model Name is : ", model_name)

elif model_type == "distilroberta":
    model_type = "roberta"
    model_name = "distilroberta-base"
    print("Your Selected Model Name is : ", model_name)

elif model_type == "electra-base":
    model_type = "electra"
    model_name = "google/electra-base-discriminator"
    print("Your Selected Model Name is : ", model_name)

elif model_type == "electra-small":
    model_type = "electra"
    model_name = "google/electra-small-discriminator"
    print("Your Selected Model Name is : ", model_name)

elif model_type == "xlnet":
    model_name = "xlnet-base-cased"
    print("Your Selected Model Name is : ", model_name)


Your Selected Model Name is :  bert-base-cased


In [9]:
wandb.init(project="Question Answer Application")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 67


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [10]:
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size = 3
model_args.num_train_epochs = 5

In [11]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    'num_train_epochs': 100,
    'learning_rate': 1e-5,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question Answer Application",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    "use_gpu": True,  # Use GPU for training
    "fp16": True,  # Use mixed-precision training
    "fp16_opt_level": "O1",  # Mixed-precision optimization level
    "gradient_accumulation_steps": 2,  # Accumulate gradients to simulate larger batch size
}

In [12]:
model = QuestionAnsweringModel(
    model_type,model_name, args=train_args,use_cuda=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [13]:
model.train_model(converted_data, eval_data='random_validation_data.json')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
add example index and unique id: 100%|██████████| 2596/2596 [00:00<00:00, 949708.96it/s]


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 1 of 100:   0%|          | 0/21 [00:00<?, ?it/s]

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

convert squad examples to features: 100%|██████████| 536/536 [00:02<00:00, 201.52it/s]

add example index and unique id: 100%|██████████| 536/536 [00:00<00:00, 473992.61it/s]


Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

Running Epoch 100 of 100:   0%|          | 0/21 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/9 [00:00<?, ?it/s]

(1000,
 {'global_step': [10,
   20,
   30,
   40,
   50,
   60,
   70,
   80,
   90,
   100,
   110,
   120,
   130,
   140,
   150,
   160,
   170,
   180,
   190,
   200,
   210,
   220,
   230,
   240,
   250,
   260,
   270,
   280,
   290,
   300,
   310,
   320,
   330,
   340,
   350,
   360,
   370,
   380,
   390,
   400,
   410,
   420,
   430,
   440,
   450,
   460,
   470,
   480,
   490,
   500,
   510,
   520,
   530,
   540,
   550,
   560,
   570,
   580,
   590,
   600,
   610,
   620,
   630,
   640,
   650,
   660,
   670,
   680,
   690,
   700,
   710,
   720,
   730,
   740,
   750,
   760,
   770,
   780,
   790,
   800,
   810,
   820,
   830,
   840,
   850,
   860,
   870,
   880,
   890,
   900,
   910,
   920,
   930,
   940,
   950,
   960,
   970,
   980,
   990,
   1000,
   1000],
  'correct': [0,
   0,
   2,
   0,
   0,
   0,
   2,
   38,
   107,
   129,
   155,
   169,
   185,
   205,
   224,
   241,
   245,
   260,
   269,
   281,
   294,
   302,
   3

In [14]:
to_predict = [
   {
    "context": "Mistborn is a series of epic fantasy novels written by American author Brandon Sanderson. The first series, published between 2006 and 2008, consists of The Final Empire, The Well of Ascension, and The Hero of Ages.",
    "qas": [
        {
            "id": "00001",
            "question": "Who is the author of the Mistborn series?",
            "answers": [
                {
                    "text": "",
                    "answer_start": -1
                }
            ]
        }
    ]
}
]

In [15]:
model.predict(to_predict=to_predict)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 2910.69it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

([{'id': '00001',
   'answer': ['Brandon Sanderson',
    'epic fantasy novels written by American author Brandon Sanderson',
    'American author Brandon Sanderson']}],
 [{'id': '00001',
   'probability': [0.9460530698619365,
    0.027473891856921295,
    0.02647303828076098]}])

In [16]:
from transformers import QuestionAnsweringPipeline, AutoModelForQuestionAnswering, AutoTokenizer

# Initialize the pipeline
model_checkpoint = f"{model_type}-base-cased"  # Adjust based on your model type
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
qa_pipeline = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

# Save the model
output_dir = f"outputs/{model_type}/trained_model"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

# Save the pipeline for later use
qa_pipeline.save_pretrained(output_dir)

print(f"Trained model saved at: {output_dir}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trained model saved at: outputs/bert/trained_model


In [17]:
"""output_dir = model.args.output_dir

# Save the model using the output directory
model.save_model(output_dir)"""

'output_dir = model.args.output_dir\n\n# Save the model using the output directory\nmodel.save_model(output_dir)'

In [19]:
import torch
from simpletransformers.question_answering import QuestionAnsweringModel
from transformers import BertTokenizer

# Load the saved model and tokenizer
saved_model_dir = "outputs/bert/trained_model"  # Adjust based on your saved model directory
model = QuestionAnsweringModel("bert", saved_model_dir)

# Create a tokenizer object
tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

# Define the context and question for which you want to generate predictions
context = "Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary."
question = "What is on top of the Main Building?"

# Tokenize the input
inputs = tokenizer(question, context, return_tensors="pt")

# Generate predictions
outputs = model.model(**inputs)

# Decode the predicted answer
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])

# Print the predicted answer
print("Predicted Answer:", answer)

Predicted Answer: Main Building? [SEP] Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of


In [20]:
# Define the context and question for which you want to generate predictions
context = "the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858."
question = "To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France??"

# Tokenize the input
inputs = tokenizer(question, context, return_tensors="pt")

# Generate predictions
outputs = model.model(**inputs)

# Decode the predicted answer
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])

# Print the predicted answer
print("Predicted Answer:", answer)

Predicted Answer: did the Virgin Mary allegedly appear in 1858 in Lourdes France?? [SEP] the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in


In [21]:
# Define the context and question for which you want to generate predictions
context = "the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858."
question = "where did the Virgin Mary allegedly appeared to Saint Bernadette Soubirous in 1858??"

# Tokenize the input
inputs = tokenizer(question, context, return_tensors="pt")

# Generate predictions
outputs = model.model(**inputs)

# Decode the predicted answer
answer_start = torch.argmax(outputs.start_logits)
answer_end = torch.argmax(outputs.end_logits) + 1
answer = tokenizer.decode(inputs["input_ids"][0][answer_start:answer_end])

# Print the predicted answer
print("Predicted Answer:", answer)

Predicted Answer: ##tte Soubirous in 1858?? [SEP] the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in
